In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("fake_and_real_news.csv")
df

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real
...,...,...
9895,Wikileaks Admits To Screwing Up IMMENSELY Wit...,Fake
9896,Trump consults Republican senators on Fed chie...,Real
9897,Trump lawyers say judge lacks jurisdiction for...,Real
9898,WATCH: Right-Wing Pastor Falsely Credits Trum...,Fake


In [6]:
df.shape

(9900, 2)

In [10]:
# Number of rows labelled as fake news

sum_rows_fake = df[df['label']=='Fake'].shape[0]

print(sum_rows_fake)

5000


In [12]:
sum_rows_real = df[df['label']=='Real'].shape[0]
print(sum_rows_real)


4900
